In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=5

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=5


In [2]:
import os
import sys

module_path = os.path.abspath(os.path.join('..'))

if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
OUT_PATH: str = '../results/manifolds/_raw'

In [4]:
DATA_PATH: str = '../data/imdb'
DATASETS: list = [
    ('train', 'sample.train'),
    ('test', 'sample.test')
]

In [5]:
DIMS: list = [768, 576, 384, 192, 96, 48, 24, 12, 6, 3]
MODELS: list = [
    ('base', 'bert-base-uncased'),
    ('textattack', 'textattack/bert-base-uncased-imdb'),
    ('fabriceyhc', 'fabriceyhc/bert-base-uncased-imdb'),
    ('wakaka', 'Wakaka/bert-finetuned-imdb')
]

In [6]:
### Load Datasets into memory

In [7]:
data_config: dict = {
    'data_label': 'text',
    'target_label': 'sentiment',
    'target_groups': {
        "negative": 0,
        "positive": 1
    }
}

In [8]:
from typing import Dict
from modules import Data

datasets: Dict[str, Data] = {
    label: Data(file_path=f"{DATA_PATH}.{name}.csv", **data_config)
    for label, name in DATASETS
}

In [9]:
for label, dataset in datasets.items():
    display(dataset.data)
    display(dataset.data['sentiment'].value_counts(normalize=True))

,text,sentiment
0,"And it falls squarely into the category of ""aw...",negative
1,This is one seriously disturbed movie. Even Th...,negative
2,"Basically this is an overlong, unfunny, action...",negative
3,Hey if you have a little over an hour to kill ...,negative
4,Did anyone read the script. This has to be som...,negative
...,...,...
1240,"First of all, Jenna Jameson is the best actres...",negative
1241,"I didnt think it was possible, but i have foun...",negative
1242,"OK, I taped this off TV and missed the very st...",negative
1243,"Okay, okay, maybe not THE greatest. I mean, Th...",positive


negative    0.503614
positive    0.496386
Name: sentiment, dtype: float64

,text,sentiment
0,This...... Movie.... Is..... Horrible!!!!!! Yo...,negative
1,At the same time John Russell was playing ranc...,positive
2,This is the best version of Gypsy that has bee...,positive
3,"It's just stories, some we wish happen to us, ...",positive
4,"This film, without doubt, is the clearest exam...",positive
...,...,...
1235,"""Cooley High"" is one of my favorite movies EVE...",positive
1236,The Comic Strip featured actors from 'The Youn...,negative
1237,I suppose you could say this film has a grain ...,negative
1238,"Having just watched Acacia, I find that I have...",negative


negative    0.520968
positive    0.479032
Name: sentiment, dtype: float64

In [10]:
### Load Encoder into Memory

In [11]:
encoder_config: dict = {
    'layers': [11]
}

In [12]:
from modules import Encoder

encoders: Dict[str, Encoder] = {
    label: Encoder({**{'model': ref}, **encoder_config})
    for label, ref in MODELS
}

In [13]:
### Compute manifolds and measure centroid distance and cluster dispersion

In [14]:
import numpy as np
import pandas as pd

In [15]:
from sklearn.manifold import MDS

def manifold_reduction(data: np.ndarray, dim: int = 3) -> np.ndarray:
    # mds translates using the pairwise distances among points
    return MDS(n_components=dim, n_jobs=-1).fit_transform(data)

In [16]:
from scipy.spatial import distance

def metric_computation(groups: pd.core.groupby.GroupBy, dim: int, col: str) -> dict:

    # hold data in dict record
    record: dict = {'dim': dim}

    # centroids, intra distance per group (binary and multi class)
    for label, group in groups:

            # calculate centroid as cluster mean
            record[f'centroid_{label}'] = np.mean(
                np.stack(
                    group[col].tolist(),
                    axis=0
                ),
                axis=0
            ).tolist()

            #  calculate distance between each cluster point and centroid
            record[f'centroid_point_distances_{label}'] = distance.cdist(
                [record[f'centroid_{label}']],
                group[col].tolist()
            ).tolist()

            # calculate intra cluster distance as centroid point distance mean
            record[f'intra_distance_{label}'] = np.mean(
                record[f'centroid_point_distances_{label}'],
                axis=1
            ).item()

    # extra distance as binary centroid distance (positive|negative)
    # TODO expand to multi class
    record['extra_distance'] = distance.cdist(
        [record['centroid_positive']],
        [record['centroid_negative']]
    ).item()

    return record

In [17]:
from typing import Generator
import torch

def reduce_analyse(data: pd.DataFrame, col: str, dim: list, default_dim: int = 768) -> Generator:

    embed_col: np.ndarray = torch.stack(data[col].tolist()).numpy()

    for d in dim:

        # if reduction size is equal to encoder output dim, skip manifold reduction
        if d == default_dim:
            data['reduced_embeds'] = list(embed_col)
        else:
            data['reduced_embeds'] = list(manifold_reduction(embed_col, dim=d))

        yield metric_computation(
            data.groupby(dataset.target_label),
            dim=d, col='reduced_embeds'
        )

In [18]:
results: Dict[str, pd.DataFrame] = {}

In [19]:
# apply each encoder to all datasets
for enc_label, encoder in encoders.items():
    for data_label, dataset in datasets.items():

        # encode text column
        encoder.df_encode(dataset.data, col=dataset.data_label)

        # apply cluster analysis and export as dataframe
        results[f'{data_label}.{enc_label}'] = pd.DataFrame.from_records(
            list(reduce_analyse(
                dataset.data, encoder.col_name, DIMS,
                default_dim=encoder.dim)
            )
        )

In [20]:
output_cols: str = '(dim|extra_distance|intra_distance_*|centroid_point_distances_*)'

In [21]:
for label, dataset in results.items():
    results[label].filter(regex=output_cols).to_csv(f'{OUT_PATH}/{label}.csv')
    display(label, results[label].filter(regex=output_cols))

'train.base'

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[5.992325561399737, 5.203575077395552, 3.5995...",5.764313,"[[6.28652306558972, 3.8467728310963367, 5.1201...",6.138071,1.259246
1,576,"[[5.998907561659804, 5.20429223352104, 3.60059...",5.765730,"[[6.306472168343806, 3.8406067732668197, 5.137...",6.139768,1.193737
2,384,"[[5.99809831110702, 5.209406302314851, 3.59817...",5.765468,"[[6.302880179398811, 3.8439582126890106, 5.129...",6.139453,1.197830
3,192,"[[5.989179558040728, 5.20308222105544, 3.58133...",5.763025,"[[6.30275337188162, 3.804601725111403, 5.13448...",6.137499,1.220880
4,96,"[[6.001490161397765, 5.200131744815813, 3.5928...",5.760449,"[[6.304338052169215, 3.8114252952024854, 5.137...",6.135308,1.237246
5,48,"[[5.978139737305378, 5.17674970095341, 3.54733...",5.752931,"[[6.275858301193788, 3.7716448119043506, 5.101...",6.128193,1.315556
6,24,"[[5.984241262573122, 5.128108147891398, 3.5290...",5.736505,"[[6.186795323682618, 3.7806063934981142, 5.018...",6.113381,1.437465
7,12,"[[5.935683877594702, 5.0894889821615195, 3.455...",5.699003,"[[6.207877198480866, 3.5430944660054875, 5.033...",6.080595,1.681827
8,6,"[[5.923193085156102, 4.869691135083833, 3.2336...",5.625003,"[[6.306060399857682, 3.335986604969422, 4.8193...",6.027260,1.801340
9,3,"[[5.85203375999411, 4.791429520515079, 2.81340...",5.429721,"[[5.9604018162228645, 2.9664268431571874, 4.80...",5.957408,1.447820


'test.base'

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[4.832424433221461, 9.807014103979695, 3.7882...",5.798742,"[[5.229993597663103, 4.427183632815254, 5.6675...",6.084546,1.229414
1,576,"[[4.855815287883087, 9.803366795820503, 3.8120...",5.800334,"[[5.239752862285097, 4.418781130039162, 5.6664...",6.086612,1.154896
2,384,"[[4.851692053966959, 9.806107043630272, 3.8091...",5.799666,"[[5.232982052660435, 4.432929383999893, 5.6582...",6.085929,1.162938
3,192,"[[4.849376566608824, 9.79537313965205, 3.81052...",5.798720,"[[5.226997282097812, 4.400307953161804, 5.6606...",6.084580,1.175887
4,96,"[[4.831381840702488, 9.806197866204979, 3.7824...",5.795773,"[[5.254985269573869, 4.406814474286478, 5.6697...",6.081798,1.201157
5,48,"[[4.789311957478701, 9.815455123641163, 3.7847...",5.788482,"[[5.203252643245383, 4.334539989235488, 5.6522...",6.073079,1.297727
6,24,"[[4.715640775269686, 9.848768446858827, 3.7014...",5.773222,"[[5.206691394231041, 4.2870024697869304, 5.635...",6.059071,1.404172
7,12,"[[4.7194338867860495, 9.895807775076673, 3.643...",5.744582,"[[5.2437577681448815, 4.287747248851521, 5.577...",6.033016,1.535413
8,6,"[[4.294540899133911, 9.97537934213856, 3.10632...",5.666244,"[[4.848189758825476, 3.6853112524572817, 5.556...",5.962391,1.810618
9,3,"[[4.281322298005763, 10.012408322920678, 2.171...",5.483858,"[[4.969311071299542, 3.948964141784532, 5.4763...",5.870369,1.508725


'train.textattack'

...

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[9.073984610715168, 10.79574947060581, 10.667...",9.273344,"[[3.9066403335518403, 9.816295601628925, 11.72...",7.438157,19.964325
1,576,"[[9.06992614970188, 10.800045351310528, 10.669...",9.271932,"[[3.8890632235390843, 9.819785380095174, 11.72...",7.436771,19.964599
2,384,"[[9.07007529164065, 10.797333588568195, 10.670...",9.271734,"[[3.8910182075364883, 9.824774339086003, 11.72...",7.435813,19.964717
3,192,"[[9.069799017971265, 10.804503100101812, 10.66...",9.271650,"[[3.887623179281154, 9.820909703245782, 11.726...",7.436544,19.964907
4,96,"[[9.066407841331793, 10.79928185824498, 10.668...",9.270937,"[[3.8733417814869577, 9.820079236729807, 11.72...",7.434083,19.965551
5,48,"[[9.075261868783645, 10.816659058285115, 10.67...",9.268299,"[[3.8833185097806364, 9.841535991094785, 11.73...",7.430381,19.966897
6,24,"[[9.066545539975548, 10.813844096999658, 10.67...",9.264909,"[[3.880122390595751, 9.83701755685778, 11.7283...",7.424555,19.970252
7,12,"[[9.100826811341541, 10.80368415005548, 10.694...",9.251533,"[[3.86162414969133, 9.850031429356708, 11.7106...",7.406760,19.979570
8,6,"[[9.141878211491996, 10.935425290244883, 10.75...",9.209127,"[[3.601098569782421, 9.900103802798562, 11.748...",7.338289,20.008512
9,3,"[[9.158822279650673, 10.232417097741667, 10.99...",8.909093,"[[3.181293716269652, 9.921919133134383, 11.491...",6.985437,20.246208


'train.wakaka'

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[8.912249447423662, 5.991112258898744, 6.4723...",7.437458,"[[11.313891151375156, 10.330851402943722, 5.53...",8.561553,11.483087
1,576,"[[8.90885134907158, 5.982331431708107, 6.46783...",7.435582,"[[11.279683472043187, 10.295851851720883, 5.53...",8.559251,11.485798
2,384,"[[8.905329441731666, 5.9824046861561335, 6.468...",7.435125,"[[11.28073226210674, 10.300748659445828, 5.534...",8.558938,11.486248
3,192,"[[8.903514197064906, 5.9841287951360185, 6.463...",7.433320,"[[11.277017410458756, 10.298353000519308, 5.53...",8.556920,11.489656
4,96,"[[8.908116408368548, 5.981449208167516, 6.4543...",7.429639,"[[11.26084998955271, 10.301450704820029, 5.500...",8.553358,11.494260
5,48,"[[8.899269267900785, 5.960281380208095, 6.4290...",7.421411,"[[11.285779032150222, 10.296455812357042, 5.46...",8.545226,11.507993
6,24,"[[8.913132017885491, 5.9277619373770305, 6.420...",7.403176,"[[11.241352431001893, 10.269065221827828, 5.38...",8.526624,11.536142
7,12,"[[8.855939630511708, 5.939909028033813, 6.2673...",7.356365,"[[11.099102818305294, 10.179133175337357, 5.37...",8.476570,11.611673
8,6,"[[8.834782650292649, 5.0154779273389325, 6.034...",7.218211,"[[10.954506302226845, 9.997661259557704, 5.112...",8.336106,11.829616
9,3,"[[8.82980719945389, 4.548188262132617, 5.76543...",6.757139,"[[10.87100169627255, 9.747964656113373, 4.1889...",7.903175,12.396740


'test.wakaka'

,dim,centroid_point_distances_negative,intra_distance_negative,centroid_point_distances_positive,intra_distance_positive,extra_distance
0,768,"[[6.595880875307386, 8.794097214610463, 7.2356...",7.581062,"[[6.3813950679018925, 9.885993432500822, 6.399...",8.446871,11.398275
1,576,"[[6.598851457776016, 8.795474086577475, 7.2321...",7.579218,"[[6.365344573924828, 9.866397399173747, 6.3815...",8.444666,11.400987
2,384,"[[6.598614119272363, 8.792584175630877, 7.2294...",7.578515,"[[6.377239684717132, 9.866619407535786, 6.3811...",8.444124,11.401848
3,192,"[[6.603452619081457, 8.794447139235105, 7.2271...",7.577133,"[[6.373549529392549, 9.86019918844127, 6.38728...",8.442543,11.403979
4,96,"[[6.593191233452919, 8.79945676555837, 7.22372...",7.573466,"[[6.362347403072847, 9.87423549608306, 6.34715...",8.438306,11.410811
5,48,"[[6.595364374107468, 8.784340553542048, 7.2265...",7.565383,"[[6.337306244462101, 9.846386478982225, 6.3443...",8.429617,11.424288
6,24,"[[6.548809308502468, 8.749814663665424, 7.1977...",7.547136,"[[6.272178324462712, 9.8272926277015, 6.311980...",8.408072,11.457864
7,12,"[[6.594984399077988, 8.757362971568737, 7.1621...",7.501272,"[[6.202472856527514, 9.764558570436673, 6.1886...",8.352838,11.537099
8,6,"[[6.426641699285559, 8.822089186706071, 7.0631...",7.373446,"[[5.6114946468036, 9.655419117545906, 6.040898...",8.210152,11.744673
9,3,"[[5.105434059748647, 8.854781247830145, 6.2779...",6.928481,"[[5.265330775801922, 9.26810628440048, 5.41939...",7.752875,12.347285
